# Integrated Networks: Analyzing Dorsal and Ventral Visual Pathway Connectivity in Object Recognition

## Background

The human brain processes visual information through two primary pathways: the ventral "what" pathway, critical for recognizing and identifying objects, and the dorsal "where/how" pathway, traditionally associated with spatial processing and action guidance. For decades, these pathways were viewed as distinct processing streams with separate functions. However, recent research suggests they may be more integrated than previously thought.

This integration matters beyond neuroscience because it provides insight into how our brain combines object recognition with action planning - a critical function for technologies like computer vision, robotics, and treatments for visual processing disorders. By analyzing functional connectivity patterns between key regions in these pathways (lateral occipital complex in the ventral stream and posterior intraparietal sulcus in the dorsal stream), this study aims to determine whether these pathways function as separate networks or as an integrated system during object recognition.

## Variables

- **Participants**: 18 right-handed individuals (10 female, mean age = 17.56 years) with normal or corrected-to-normal vision
- **Data Type**: Functional Magnetic Resonance Imaging (fMRI) data
- **Regions of Interest (ROIs)**: 
  - **Lateral Occipital Complex (LO)**: Categorical, ventral "what" pathway region
  - **Posterior Intraparietal Sulcus (pIPS)**: Categorical, dorsal "where/how" pathway region
- **Connectivity Measures**: 
  - **Dice Coefficient**: Continuous (0-1), measuring spatial overlap between connectivity networks
  - **Connectivity Fingerprints**: Continuous values, correlation coefficients between ROI time series and atlas regions
  - **Vector Correlations**: Continuous (-1 to 1), correlation between connectivity profiles
- **Data Acquisition Parameters**:
  - 69 slices
  - TR = 2000ms
  - TE = 30ms
  - Flip angle = 79°
  - Voxel size = 2×2×2mm³
  - FOV = 212mm
  - Multi-band acceleration factor = 3

## Hypotheses

We hypothesize that dorsal (pIPS) and ventral (LO) visual pathways function as a single integrated network rather than two separate networks during object recognition. We can express this through the following specific hypotheses and models:

1. **Spatial Overlap Hypothesis**: Within-subject spatial overlap between dorsal and ventral networks will be significantly higher than between-subject overlap for either network.
   - Model: $\text{Dice}_{\text{within-subject}} > \text{Dice}_{\text{between-subject}}$
   - Statistical test: Repeated measures ANOVA comparing Dice coefficients

2. **Connectivity Profile Hypothesis**: Despite functioning as an integrated network, we predict distinct connectivity profiles for each pathway, with the dorsal pathway showing more extensive connectivity.
   - Model: $\text{Number of connections}_{\text{pIPS}} \neq \text{Number of connections}_{\text{LO}}$
   - Statistical test: Chi-square test for connection count asymmetry

3. **Connection Strength Hypothesis**: While connection count may differ, we predict similar connection strengths for both pathways.
   - Model: $\text{Mean strength}_{\text{pIPS}} \approx \text{Mean strength}_{\text{LO}}$
   - Statistical test: Mann-Whitney U test comparing connection strength distributions

## Data Organization

The data for this analysis was organized in the following structure:

- **Raw Data**: fMRI time series for 18 participants performing an object processing task
- **Processed Data**: 
  - ROI-based functional connectivity maps in MNI space
  - Left and right hemisphere data for each ROI (pIPS and LO)
  - Atlas-based connectivity measures using a merged Schaefer-Wang atlas
  
**Final Data Table Structure:**

| Subject ID | ROI | Hemisphere | Region | Connectivity Value |
|------------|-----|------------|--------|-------------------|
| sub-001    | pIPS | Left      | Region1| 0.42              |
| sub-001    | pIPS | Right     | Region1| 0.39              |
| sub-001    | LO   | Left      | Region1| 0.36              |
| ...        | ...  | ...       | ...    | ...               |

This organization allows for:
1. Within-subject comparisons between ROIs (pIPS vs. LO)
2. Between-subject comparisons within ROIs
3. Hemisphere-specific and bilateral analyses
4. Region-by-region connectivity fingerprint analysis

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import nibabel as nib
from scipy import stats
import matplotlib.pyplot as plt
from nilearn import image, datasets, plotting
from nilearn.maskers import NiftiLabelsMasker
from scipy.stats import pearsonr
from sklearn.utils import resample

# Define base paths
base_dir = '/user_data/csimmon2/git_repos/ptoc'
roi_dir = f'{base_dir}/roiParcels'
results_dir = f'{base_dir}/results'
group_out_dir = f'{results_dir}/group_averages'
study_dir = "/lab_data/behrmannlab/vlad/ptoc"
sub_info_path = '/user_data/csimmon2/git_repos/ptoc/sub_info.csv'

# Create output directories
os.makedirs(results_dir, exist_ok=True)
os.makedirs(group_out_dir, exist_ok=True)

# Define analysis-specific directories
connectivity_dir = f'{results_dir}/connectivity_comparison'
dice_output_dir = f'{results_dir}/dice_comparison'
asymmetry_output_dir = f'{results_dir}/connectivity_asymmetry'

# Create analysis-specific directories
os.makedirs(connectivity_dir, exist_ok=True)
os.makedirs(dice_output_dir, exist_ok=True)
os.makedirs(asymmetry_output_dir, exist_ok=True)

## Analysis

Our analysis employed three primary methods to investigate the network organization of object recognition in the human brain:

### 1. Dice Coefficient Analysis

We used Dice coefficients to quantify the spatial overlap between connectivity networks, allowing us to directly test our spatial overlap hypothesis. The Dice coefficient is calculated as:

$$\text{Dice} = \frac{2|X \cap Y|}{|X| + |Y|}$$

where $X$ and $Y$ represent the binarized connectivity maps being compared.

This analysis revealed remarkably high within-subject overlap between dorsal (pIPS) and ventral (LO) networks (mean Dice coefficient = 0.91), compared to between-subject overlap for either dorsal pIPS (0.83) or ventral LO (0.79) regions. 

Statistical analysis using repeated measures ANOVA confirmed significant differences among the three comparison types (F = 33.54, p < 0.001), with post-hoc tests confirming that within-subject overlap between pIPS and LO is significantly higher than between-subject overlap for either pIPS or LO (all p-values < 0.01).

In [ ]:
# Dice Coefficient Analysis Script
def dice_coefficient(img1_data, img2_data):
    """
    Calculate Dice coefficient between two binary arrays.
    
    Parameters:
    -----------
    img1_data : numpy.ndarray
        First binary image data
    img2_data : numpy.ndarray
        Second binary image data
        
    Returns:
    --------
    float
        Dice coefficient value between 0 (no overlap) and 1 (perfect overlap)
    """
    # Ensure binary
    img1_bin = (img1_data > 0).astype(int)
    img2_bin = (img2_data > 0).astype(int)
    
    # Calculate intersection
    intersection = np.sum(img1_bin * img2_bin)
    
    # Calculate Dice using formula: 2*intersection/(sum of elements)
    dice = 2.0 * intersection / (np.sum(img1_bin) + np.sum(img2_bin))
    
    return dice

def analyze_data_type(analysis_type):
    """
    Analyze dice coefficients for a specific data type (fc or ppi)
    
    Parameters:
    -----------
    analysis_type : str
        Type of analysis to perform ('fc' for functional connectivity)
        
    Returns:
    --------
    dict
        Dictionary containing results of the analysis
    """
    print(f"\n{'='*80}")
    print(f"ANALYZING {analysis_type.upper()} MAPS")
    print(f"{'='*80}")
    
    # Load subject info
    sub_info = pd.read_csv(sub_info_path)
    subjects = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    
    # Exclude sub-084 as specified
    if 'sub-084' in subjects:
        subjects.remove('sub-084')
        print("Excluded sub-084 from analysis")
    
    print(f"Found {len(subjects)} control subjects")
    
    # Define ROIs and hemispheres
    rois = ['pIPS', 'LO']
    hemispheres = ['left', 'right']
    
    # Load and preprocess subject data
    subject_data = {}
    valid_subjects = []
    
    for sub in subjects:
        has_all_data = True
        subject_data[sub] = {}
        
        for roi in rois:
            # Initialize lists to store hemisphere data
            roi_data_arrays = []
            
            for hemi in hemispheres:
                # Both FC and PPI files are in the same directory but with different analysis_type in the filename
                roi_file = f'{study_dir}/{sub}/ses-01/derivatives/fc_mni/{sub}_{roi}_{hemi}_loc_{analysis_type}_mni.nii.gz'
                
                if os.path.exists(roi_file):
                    try:
                        img = nib.load(roi_file)
                        roi_data_arrays.append(img.get_fdata())
                    except Exception as e:
                        print(f"Error loading {roi_file}: {e}")
                        has_all_data = False
                else:
                    print(f"File not found: {roi_file}")
                    has_all_data = False
            
            # Average the hemispheres if we have data for both
            if len(roi_data_arrays) == 2:
                # Create averaged map
                avg_data = (roi_data_arrays[0] + roi_data_arrays[1]) / 2
                subject_data[sub][roi] = avg_data
            else:
                has_all_data = False
        
        if has_all_data:
            valid_subjects.append(sub)
    
    print(f"Found {len(valid_subjects)} subjects with complete {analysis_type} data")
    
    if len(valid_subjects) == 0:
        print(f"No valid subjects found for {analysis_type} analysis. Skipping.")
        return None
    
    # 1. Within-subject, between-ROI analysis (pIPS vs LO)
    print(f"\nCalculating within-subject, between-ROI dice coefficients for {analysis_type}...")
    between_roi_results = []
    
    for sub in valid_subjects:
        try:
            # Calculate Dice coefficient
            dice_score = dice_coefficient(subject_data[sub]['pIPS'], subject_data[sub]['LO'])
            
            between_roi_results.append({
                'Subject': sub,
                'Analysis': analysis_type,
                'Dice': dice_score
            })
            
            print(f"Subject {sub}, {analysis_type}, pIPS vs LO: {dice_score:.4f}")
        except Exception as e:
            print(f"Error processing subject {sub}: {e}")
    
    # Convert to DataFrame
    between_roi_df = pd.DataFrame(between_roi_results)
    
    # Calculate statistics
    mean = between_roi_df['Dice'].mean()
    std = between_roi_df['Dice'].std()
    # 95% confidence interval
    ci = stats.t.interval(0.95, len(between_roi_df)-1, loc=mean, scale=std/np.sqrt(len(between_roi_df)))
    
    print(f"\n{analysis_type.upper()}: pIPS vs LO within-subject similarity:")
    print(f"Mean Dice: {mean:.4f} ± {std:.4f}")
    print(f"95% CI: [{ci[0]:.4f}, {ci[1]:.4f}]")
    print(f"N = {len(between_roi_df)} subjects")
    
    # 2. Between-subject, within-ROI analysis
    print(f"\nCalculating between-subject, within-ROI dice coefficients for {analysis_type}...")
    within_roi_results = []
    
    for roi in rois:
        # Compare each subject to all others
        for i, sub1 in enumerate(valid_subjects):
            for j, sub2 in enumerate(valid_subjects):
                if i < j:  # Only compare each pair once
                    try:
                        # Calculate Dice coefficient
                        dice_score = dice_coefficient(subject_data[sub1][roi], subject_data[sub2][roi])
                        
                        within_roi_results.append({
                            'ROI': roi,
                            'Analysis': analysis_type,
                            'Subject1': sub1,
                            'Subject2': sub2,
                            'Dice': dice_score
                        })
                    except Exception as e:
                        print(f"Error comparing {sub1} and {sub2} for {roi}: {e}")
    
    # Convert to DataFrame
    within_roi_df = pd.DataFrame(within_roi_results)
    
    # Calculate statistics for each ROI
    within_roi_stats = []
    
    for roi in rois:
        roi_data = within_roi_df[within_roi_df['ROI'] == roi]
        
        if len(roi_data) > 0:
            mean = roi_data['Dice'].mean()
            std = roi_data['Dice'].std()
            # 95% confidence interval
            ci = stats.t.interval(0.95, len(roi_data)-1, loc=mean, scale=std/np.sqrt(len(roi_data)))
            
            within_roi_stats.append({
                'ROI': roi,
                'Analysis': analysis_type,
                'Mean Dice': mean,
                'Std Dev': std,
                '95% CI Lower': ci[0],
                '95% CI Upper': ci[1],
                'N': len(roi_data)
            })
            
            print(f"\n{analysis_type.upper()}: {roi} between-subject similarity:")
            print(f"Mean Dice: {mean:.4f} ± {std:.4f}")
            print(f"95% CI: [{ci[0]:.4f}, {ci[1]:.4f}]")
            print(f"Based on {len(roi_data)} comparisons among {len(valid_subjects)} subjects")
    
    # Save results to CSV files
    between_roi_df.to_csv(f'{output_dir}/{analysis_type}_between_roi_dice_by_subject.csv', index=False)
    within_roi_df.to_csv(f'{output_dir}/{analysis_type}_within_roi_dice_by_pair.csv', index=False)
    
    within_roi_stats_df = pd.DataFrame(within_roi_stats)
    within_roi_stats_df.to_csv(f'{output_dir}/{analysis_type}_within_roi_stats.csv', index=False)
    
    # Prepare summary data
    between_roi_stats = {
        'ROI': 'pIPS vs LO',
        'Analysis': analysis_type,
        'Mean Dice': between_roi_df['Dice'].mean(),
        'Std Dev': between_roi_df['Dice'].std(),
        '95% CI Lower': ci[0],
        '95% CI Upper': ci[1],
        'N': len(between_roi_df)
    }
    
    # Combine all stats into one DataFrame
    all_stats = pd.DataFrame([between_roi_stats] + within_roi_stats)
    
    return {
        'between_roi': between_roi_df,
        'within_roi': within_roi_df,
        'stats': all_stats
    }

def main():
    """
    Analyze FC data and calculate summary statistics
    """
    results = {}
    all_stats = []
    
    # Analyze FC data
    print("\nAnalyzing FC data...")
    fc_results = analyze_data_type('fc')
    if fc_results:
        results['fc'] = fc_results
        all_stats.append(fc_results['stats'])
    
    # Combine stats from the analysis
    if all_stats:
        combined_stats = pd.concat(all_stats)
        combined_stats.to_csv(f'{output_dir}/combined_dice_stats.csv', index=False)
        
        # Print combined summary
        print("\n\nCOMBINED SUMMARY OF DICE COEFFICIENT ANALYSES")
        print("=" * 80)
        print(combined_stats.to_string(index=False))
    
    return results

# Run the Dice coefficient analysis
results = main()

In [ ]:
# Compare within-ROI vs between-ROI dice coefficients using ANOVA
def compare_within_vs_between_dice_anova_averaged(analysis_type='fc'):
    """
    Compare within-ROI vs between-ROI dice coefficients using ANOVA with subject-level averages
    
    Parameters:
    -----------
    analysis_type : str
        Type of analysis to perform ('fc' for functional connectivity)
        
    Returns:
    --------
    dict
        Dictionary containing results of the ANOVA analysis
    """
    # Define study directories
    study_dir = "/lab_data/behrmannlab/vlad/ptoc"
    results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
    sub_info_path = '/user_data/csimmon2/git_repos/ptoc/sub_info.csv'
    output_dir = f'{results_dir}/dice_comparison'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Comparing within-ROI vs between-ROI dice coefficients using ANOVA with subject averages for {analysis_type.upper()}...")
    
    # Load subject info
    sub_info = pd.read_csv(sub_info_path)
    subjects = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    
    # Exclude sub-084 as specified
    if 'sub-084' in subjects:
        subjects.remove('sub-084')
        print("Excluded sub-084 from analysis")
    
    print(f"Found {len(subjects)} control subjects")
    
    # Define ROIs and hemispheres
    rois = ['pIPS', 'LO']
    hemispheres = ['left', 'right']
    
    # Load and preprocess subject data
    subject_data = {}
    valid_subjects = []
    
    for sub in subjects:
        has_all_data = True
        subject_data[sub] = {}
        
        for roi in rois:
            # Initialize lists to store hemisphere data
            roi_data_arrays = []
            
            for hemi in hemispheres:
                roi_file = f'{study_dir}/{sub}/ses-01/derivatives/fc_mni/{sub}_{roi}_{hemi}_loc_{analysis_type}_mni.nii.gz'
                
                if os.path.exists(roi_file):
                    try:
                        img = nib.load(roi_file)
                        roi_data_arrays.append(img.get_fdata())
                    except Exception as e:
                        print(f"Error loading {roi_file}: {e}")
                        has_all_data = False
                else:
                    print(f"File not found: {roi_file}")
                    has_all_data = False
            
            # Average the hemispheres if we have data for both
            if len(roi_data_arrays) == 2:
                # Create averaged map
                avg_data = (roi_data_arrays[0] + roi_data_arrays[1]) / 2
                subject_data[sub][roi] = (avg_data > 0).astype(int)  # Binarize
            else:
                has_all_data = False
        
        if has_all_data:
            valid_subjects.append(sub)
    
    print(f"Found {len(valid_subjects)} subjects with complete {analysis_type} data")
    
    if len(valid_subjects) == 0:
        print(f"No valid subjects found for {analysis_type} analysis. Skipping.")
        return None
    
    # 1. Within-subject, between-ROI analysis (pIPS vs LO)
    print(f"\nCalculating within-subject, between-ROI dice coefficients for {analysis_type}...")
    between_roi_results = []
    
    for sub in valid_subjects:
        try:
            # Get binary maps
            pips_bin = subject_data[sub]['pIPS']
            lo_bin = subject_data[sub]['LO']
            
            # Calculate intersection
            intersection = np.sum(pips_bin * lo_bin)
            
            # Calculate Dice
            dice_score = 2.0 * intersection / (np.sum(pips_bin) + np.sum(lo_bin))
            
            between_roi_results.append({
                'Subject': sub,
                'Dice': dice_score,
                'Comparison': 'Within-subject_between-ROI'
            })
            
            print(f"Subject {sub}, pIPS vs LO: {dice_score:.4f}")
        except Exception as e:
            print(f"Error processing subject {sub}: {e}")
    
    # 2. Calculate all pairwise between-subject dice coefficients
    print(f"\nCalculating pairwise between-subject, within-ROI dice coefficients for {analysis_type}...")
    pairwise_results = {}
    
    for roi in rois:
        pairwise_results[roi] = {}
        # Initialize the nested dictionary structure
        for sub in valid_subjects:
            pairwise_results[roi][sub] = {}
        
        # Compare each subject to all others
        for i, sub1 in enumerate(valid_subjects):
            for j, sub2 in enumerate(valid_subjects):
                if i < j:  # Only compare each pair once
                    try:
                        # Get binary maps
                        map1 = subject_data[sub1][roi]
                        map2 = subject_data[sub2][roi]
                        
                        # Calculate intersection
                        intersection = np.sum(map1 * map2)
                        
                        # Calculate Dice
                        dice_score = 2.0 * intersection / (np.sum(map1) + np.sum(map2))
                        
                        # Store the result for both subjects
                        pairwise_results[roi][sub1][sub2] = dice_score
                        pairwise_results[roi][sub2][sub1] = dice_score
                    except Exception as e:
                        print(f"Error comparing {sub1} and {sub2} for {roi}: {e}")
    
    # 3. Average the between-subject dice coefficients for each subject
    print(f"\nAveraging between-subject dice coefficients for each subject for {analysis_type}...")
    average_results = []
    
    for roi in rois:
        for sub in pairwise_results[roi]:
            # Calculate the average dice with all other subjects
            if len(pairwise_results[roi][sub]) > 0:  # Ensure there are values to average
                avg_dice = np.mean(list(pairwise_results[roi][sub].values()))
                
                average_results.append({
                    'Subject': sub,
                    'ROI': roi,
                    'Dice': avg_dice,
                    'Comparison': f'Between-subject_within-{roi}'
                })
                print(f"Subject {sub}, average {roi} similarity with other subjects: {avg_dice:.4f}")
    
    # Combine results for ANOVA
    all_results = pd.DataFrame(between_roi_results)
    average_df = pd.DataFrame(average_results)
    
    # Create separate DataFrames for pIPS and LO
    pips_df = average_df[average_df['ROI'] == 'pIPS'].copy()
    lo_df = average_df[average_df['ROI'] == 'LO'].copy()
    
    # Calculate summary statistics
    print(f"\nSummary Statistics for {analysis_type.upper()}:")
    
    groups = {
        'Within-subject, between-ROI': all_results['Dice'].values,
        'Between-subject, within-pIPS': pips_df['Dice'].values,
        'Between-subject, within-LO': lo_df['Dice'].values
    }
    
    for group_name, values in groups.items():
        if len(values) > 0:
            mean = np.mean(values)
            std = np.std(values)
            ci = stats.t.interval(0.95, len(values)-1, loc=mean, scale=std/np.sqrt(len(values)))
            print(f"{group_name}:")
            print(f"  N = {len(values)}")
            print(f"  Mean = {mean:.4f} ± {std:.4f}")
            print(f"  95% CI = [{ci[0]:.4f}, {ci[1]:.4f}]")
    
    # Save summary statistics to CSV
    summary_stats = []
    for group_name, values in groups.items():
        if len(values) > 0:
            mean = np.mean(values)
            std = np.std(values)
            ci = stats.t.interval(0.95, len(values)-1, loc=mean, scale=std/np.sqrt(len(values)))
            summary_stats.append({
                'Analysis': analysis_type,
                'Group': group_name,
                'N': len(values),
                'Mean': mean,
                'Std': std,
                'CI_Lower': ci[0],
                'CI_Upper': ci[1]
            })
    
    summary_df = pd.DataFrame(summary_stats)
    summary_df.to_csv(f'{output_dir}/{analysis_type}_anova_summary_stats.csv', index=False)
    
    # Prepare data for ANOVA - now all groups should have the same sample size based on valid subjects
    anova_data = pd.DataFrame({
        'Dice': np.concatenate([
            all_results['Dice'].values,
            pips_df['Dice'].values,
            lo_df['Dice'].values
        ]),
        'Group': np.concatenate([
            ['Within-subject_between-ROI'] * len(all_results),
            ['Between-subject_within-pIPS'] * len(pips_df),
            ['Between-subject_within-LO'] * len(lo_df)
        ])
    })
    
    # Run One-way ANOVA
    print(f"\nPerforming One-way ANOVA with subject averages for {analysis_type}...")
    import statsmodels.api as sm
    from statsmodels.formula.api import ols
    
    model = ols('Dice ~ C(Group)', data=anova_data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    
    print("\nANOVA Results:")
    print(anova_table)
    
    # Save ANOVA results to CSV
    anova_df = pd.DataFrame(anova_table)
    anova_df.to_csv(f'{output_dir}/{analysis_type}_anova_results.csv')
    
    # Calculate pairwise t-tests and apply Holm-Bonferroni correction
    unique_groups = anova_data['Group'].unique()
    p_values = []
    pairs = []

    for i, group1 in enumerate(unique_groups):
        for j, group2 in enumerate(unique_groups):
            if i < j:  # Compare each pair only once
                data1 = anova_data[anova_data['Group'] == group1]['Dice']
                data2 = anova_data[anova_data['Group'] == group2]['Dice']
                _, p_val = stats.ttest_ind(data1, data2, equal_var=False)
                p_values.append(p_val)
                pairs.append((group1, group2))

    # Apply Holm-Bonferroni correction
    from statsmodels.stats.multitest import multipletests
    reject, p_corrected, _, _ = multipletests(p_values, alpha=0.05, method='holm')

    # Print results
    print("\nHolm-Bonferroni Results:")
    for i, ((group1, group2), p_corr, rej) in enumerate(zip(pairs, p_corrected, reject)):
        print(f"{group1} vs {group2}: p-corrected = {p_corr:.4f}, significant: {rej}")

    # Save results to a text file
    with open(f'{output_dir}/{analysis_type}_holm_bonferroni_results.txt', 'w') as f:
        f.write("Holm-Bonferroni corrected pairwise comparisons:\n\n")
        for i, ((group1, group2), p_corr, rej) in enumerate(zip(pairs, p_corrected, reject)):
            f.write(f"{group1} vs {group2}: p-corrected = {p_corr:.4f}, significant: {rej}\n")
    
    # Print interpretation
    print(f"\nINTERPRETATION for {analysis_type.upper()}:")
    
    # Get F-value and p-value from ANOVA table
    f_value = anova_table.loc['C(Group)', 'F']
    p_value = anova_table.loc['C(Group)', 'PR(>F)']
    
    if p_value < 0.05:
        print(f"The ANOVA shows significant differences in Dice coefficients among the three comparison types (F={f_value:.4f}, p={p_value:.4f}).")
        print("The Holm Bonferroni post-hoc test shows which specific groups differ from each other.")
        print("\nBased on the means:")
        
        means = {
            'Within-subject_between-ROI': np.mean(groups['Within-subject, between-ROI']),
            'Between-subject_within-pIPS': np.mean(groups['Between-subject, within-pIPS']),
            'Between-subject_within-LO': np.mean(groups['Between-subject, within-LO'])
        }
        
        ordered_groups = sorted(means.items(), key=lambda x: x[1], reverse=True)
        
        for i, (group, mean) in enumerate(ordered_groups):
            print(f"{i+1}. {group}: {mean:.4f}")
        
        if means['Within-subject_between-ROI'] > means['Between-subject_within-pIPS'] and means['Within-subject_between-ROI'] > means['Between-subject_within-LO']:
            print("\nThis suggests that different ROIs within the same subject share more similar connectivity patterns")
            print("than the same ROI across different subjects. This pattern holds for both pIPS and LO.")
        else:
            print("\nThe pattern suggests a more complex relationship. Please refer to the Holm-Bon results for specific group differences.")
    else:
        print(f"The ANOVA shows no significant differences in Dice coefficients among the three comparison types (F={f_value:.4f}, p={p_value:.4f}).")
        print("This suggests that the level of similarity does not significantly differ between:")
        print("- different ROIs within the same subject")
        print("- the same ROI across different subjects")
    
    return {
        'anova_table': anova_table,
        'holm_result': {
            'pairs': pairs,
            'p_corrected': p_corrected,
            'reject': reject
        },
        'group_means': {
            'Within-subject_between-ROI': np.mean(groups['Within-subject, between-ROI']),
            'Between-subject_within-pIPS': np.mean(groups['Between-subject, within-pIPS']),
            'Between-subject_within-LO': np.mean(groups['Between-subject, within-LO'])
        }
    }

# Run ANOVA comparison for FC data
fc_results = compare_within_vs_between_dice_anova_averaged('fc')

In [ ]:
# Create Merged Atlas
base_dir = '/user_data/csimmon2/git_repos/ptoc'
roi_dir = f'{base_dir}/roiParcels'
results_dir = f'{base_dir}/results'
group_out_dir = f'{results_dir}/group_averages'
os.makedirs(results_dir, exist_ok=True)
os.makedirs(connectivity_dir, exist_ok=True)


def create_merged_atlas():
    """
    Create a merged atlas where Wang ROIs replace overlapping regions in Schaefer atlas
    
    Returns:
    --------
    dict
        Dictionary containing merged atlas information
    """
    print("\n=== Step 1: Creating Merged Atlas ===")
    
    # Load Wang ROIs - pIPS and LO
    roi_files = {
        'pIPS': f'{roi_dir}/pIPS.nii.gz',
        'LO': f'{roi_dir}/LO.nii.gz'
    }
    
    rois = {}
    for roi_name, roi_path in roi_files.items():
        if os.path.exists(roi_path):
            rois[roi_name] = nib.load(roi_path)
            print(f"Loaded {roi_name} ROI")
        else:
            print(f"Error: ROI file {roi_path} not found!")
            return None
    
    # Load Schaefer atlas
    atlas = datasets.fetch_atlas_schaefer_2018(n_rois=200, yeo_networks=7, resolution_mm=2)
    atlas_img = nib.load(atlas.maps)
    atlas_labels = atlas.labels
    print(f"Loaded Schaefer atlas with {len(atlas_labels)} parcels")
    
    # Get atlas data
    atlas_data = atlas_img.get_fdata()
    modified_atlas_data = atlas_data.copy()
    
    # Create a dictionary to store new labels
    new_labels = list(atlas_labels)
    
    # Assign values for new ROIs (continuing from the end of the Schaefer atlas)
    roi_values = {'pIPS': 201, 'LO': 202}
    overlap_info = {}
    
    # Process each ROI
    for roi_name, roi_img in rois.items():
        # Get ROI data and create mask
        roi_data = roi_img.get_fdata()
        roi_mask = roi_data > 0
        
        # Find overlapping parcels
        overlap_mask = (atlas_data > 0) & roi_mask
        overlapping_labels = np.unique(atlas_data[overlap_mask])
        overlapping_labels = overlapping_labels[overlapping_labels > 0]
        
        # Get number of voxels in overlap
        overlap_voxels = {}
        for label in overlapping_labels:
            label_mask = (atlas_data == label) & roi_mask
            overlap_voxels[int(label)] = np.sum(label_mask)
        
        # Store overlap information
        overlap_info[roi_name] = {
            'overlapping_labels': overlapping_labels.tolist(),
            'overlap_voxels': overlap_voxels
        }
        
        print(f"{roi_name} overlaps with {len(overlapping_labels)} atlas parcels")
        for label, voxels in overlap_voxels.items():
            label_idx = int(label) - 1  # Convert to 0-indexed
            if 0 <= label_idx < len(atlas_labels):
                label_name = atlas_labels[label_idx]
                label_name = label_name.decode('utf-8') if isinstance(label_name, bytes) else str(label_name)
                print(f"  Label {label} ({label_name}): {voxels} voxels")
        
        # Remove overlapping parcels from the atlas
        for label in overlapping_labels:
            label_mask = (modified_atlas_data == label) & roi_mask
            modified_atlas_data[label_mask] = 0
        
        # Add ROI with new label
        modified_atlas_data[roi_mask] = roi_values[roi_name]
        
        # Add new label name
        new_labels.append(f"Wang_{roi_name}")
    
    # Create the modified atlas
    modified_atlas_img = nib.Nifti1Image(modified_atlas_data, atlas_img.affine, atlas_img.header)
    merged_atlas_file = f'{results_dir}/schaefer_wang_merged.nii.gz'
    nib.save(modified_atlas_img, merged_atlas_file)
    print(f"Saved merged atlas to: {merged_atlas_file}")
    
    # Save new labels array
    np.save(f'{results_dir}/merged_atlas_labels.npy', new_labels)
    
    # Return atlas information
    return {
        'atlas_img': atlas_img,
        'merged_atlas_img': modified_atlas_img,
        'atlas_labels': atlas_labels,
        'merged_labels': new_labels,
        'roi_values': roi_values,
        'overlap_info': overlap_info
    }

# Create the merged atlas
merged_atlas = create_merged_atlas()

### 2. Connectivity Fingerprint Analysis

To characterize the broader network organization, we implemented a connectivity fingerprint analysis examining connectivity between our seed regions and all regions in our merged atlas. This approach revealed:

- High mean connectivity vector correlation between pIPS and LO (0.724 ± 0.222), indicating overall similar connectivity patterns
- 94 regions showing significant differences via bootstrap analysis (95% confidence intervals)
- 67 regions with high reliability via Leave-One-Out Cross-Validation
- 60 regions significant with combined criteria (meeting both bootstrap and LOOCV thresholds)

These results suggest that while the dorsal and ventral pathways have largely similar connectivity profiles (high correlation), they maintain distinct connections with specific brain regions, reflecting their specialized processing roles.

In [ ]:
# Fingerprint connectivity analysis using the merged atlas
def analyze_connectivity_and_save_results(analysis_type='fc'):
    """
    Analyze connectivity and save results as CSV using merged atlas
    
    Parameters:
    -----------
    analysis_type : str
        Type of analysis to perform ('fc' for functional connectivity)
        
    Returns:
    --------
    tuple
        DataFrame with results and dictionary with ROI data for visualization
    """
    print(f"\n{'='*80}")
    print(f"ANALYZING {analysis_type.upper()} CONNECTIVITY AND SAVING RESULTS")
    print(f"{'='*80}")
    
    # Load merged atlas instead of fetching standard Schaefer atlas
    merged_atlas_file = f'{results_dir}/schaefer_wang_merged.nii.gz'
    merged_labels_file = f'{results_dir}/merged_atlas_labels.npy'
    
    if not os.path.exists(merged_atlas_file) or not os.path.exists(merged_labels_file):
        print("Error: Merged atlas files not found. Please run merge_atlas.py first.")
        return None
    
    atlas_img = nib.load(merged_atlas_file)
    atlas_labels = np.load(merged_labels_file, allow_pickle=True)
    
    print(f"Loaded merged atlas with {len(atlas_labels)} parcels")
    
    # Load subject info
    sub_info = pd.read_csv(sub_info_path)
    subjects = sub_info[sub_info['group'] == 'control']['sub'].tolist()
    
    # Exclude sub-084 as specified
    if 'sub-084' in subjects:
        subjects.remove('sub-084')
        print("Excluded sub-084 from analysis")
        
    print(f"Found {len(subjects)} control subjects")
    
    # Define ROIs and hemispheres
    rois = ['pIPS', 'LO']
    hemispheres = ['left', 'right']
    
    # Setup atlas masker
    masker = NiftiLabelsMasker(labels_img=atlas_img, standardize=False)
    
    # Load and process subject data
    subject_data = []
    for sub in subjects:
        sub_conn = {}
        
        for roi in rois:
            # Initialize arrays to hold combined data
            combined_data = None
            hemi_count = 0
            
            for hemisphere in hemispheres:
                fc_file = f'{study_dir}/{sub}/ses-01/derivatives/fc_mni/{sub}_{roi}_{hemisphere}_loc_{analysis_type}_mni.nii.gz'
                
                if os.path.exists(fc_file):
                    try:
                        # Load the FC map
                        fc_img = nib.load(fc_file)
                        
                        # Extract ROI values using atlas
                        fc_values = masker.fit_transform(fc_img)[0]
                        
                        # Add to combined data
                        if combined_data is None:
                            combined_data = fc_values
                        else:
                            combined_data += fc_values
                        
                        hemi_count += 1
                    except Exception as e:
                        print(f"Error processing {fc_file}: {e}")
            
            # Average the data if we have at least one hemisphere
            if hemi_count > 0:
                sub_conn[roi] = combined_data / hemi_count
        
        # Only include subjects with both ROIs
        if len(sub_conn) == len(rois):
            subject_data.append(sub_conn)
    
    n_valid_subjects = len(subject_data)
    print(f"Successfully loaded data for {n_valid_subjects} subjects")
    
    if n_valid_subjects < 3:
        print("Not enough valid subjects. Analysis cannot proceed.")
        return None
    
    # Initialize arrays for analysis
    n_rois = len(atlas_labels)
    pips_fc = np.zeros((n_valid_subjects, n_rois))
    lo_fc = np.zeros((n_valid_subjects, n_rois))
    
    # Fill arrays with connectivity data
    for i, subj_data in enumerate(subject_data):
        pips_fc[i, :] = subj_data['pIPS']
        lo_fc[i, :] = subj_data['LO']
    
    # Calculate connectivity vector correlation for each subject
    vector_correlations = np.zeros(n_valid_subjects)
    for i in range(n_valid_subjects):
        vector_correlations[i] = np.corrcoef(pips_fc[i, :], lo_fc[i, :])[0, 1]
    
    print(f"Mean connectivity vector correlation: {np.mean(vector_correlations):.3f} ± {np.std(vector_correlations):.3f}")
    
    # Calculate mean connectivity profiles
    mean_pips = np.mean(pips_fc, axis=0)
    mean_lo = np.mean(lo_fc, axis=0)
    
    # Remove self-connectivity
    # Get indices for Wang ROIs in the merged atlas
    wang_pips_idx = next((i for i, label in enumerate(atlas_labels) if 'Wang_pIPS' in str(label)), None)
    wang_lo_idx = next((i for i, label in enumerate(atlas_labels) if 'Wang_LO' in str(label)), None)
    
    # Print which indices are being excluded
    if wang_pips_idx is not None:
        print(f"Excluding Wang_pIPS (index {wang_pips_idx}) from connectivity analysis")
    if wang_lo_idx is not None:
        print(f"Excluding Wang_LO (index {wang_lo_idx}) from connectivity analysis")
    
    # Create mask for non-self connections (all True except at Wang ROI indices)
    mask = np.ones(len(atlas_labels), dtype=bool)
    if wang_pips_idx is not None:
        mask[wang_pips_idx] = False
    if wang_lo_idx is not None:
        mask[wang_lo_idx] = False
    
    # Apply mask to connectivity data
    pips_fc_masked = pips_fc[:, mask]
    lo_fc_masked = lo_fc[:, mask]
    
    # Get the masked atlas labels
    atlas_labels_masked = [label for i, label in enumerate(atlas_labels) if mask[i]]
    
    # Update mean connectivity with masked data
    mean_pips_masked = np.mean(pips_fc_masked, axis=0)
    mean_lo_masked = np.mean(lo_fc_masked, axis=0)
    
    # Get the new number of ROIs after masking
    n_masked_rois = np.sum(mask)
    print(f"Analyzing {n_masked_rois} ROIs after excluding Wang ROIs")
    
    # Calculate difference profile with masked data
    diff_profile = mean_pips_masked - mean_lo_masked
    abs_diff_profile = np.abs(diff_profile)
    
    # Run bootstrap analysis with masked data
    print("Performing bootstrap analysis...")
    n_boots = 10000
    boot_diffs = np.zeros((n_boots, n_masked_rois))
    
    for i in range(n_boots):
        # Resample subjects with replacement
        boot_idx = resample(range(n_valid_subjects), replace=True, n_samples=n_valid_subjects)
        
        # Calculate mean difference for this bootstrap sample
        boot_pips = np.mean(pips_fc_masked[boot_idx, :], axis=0)
        boot_lo = np.mean(lo_fc_masked[boot_idx, :], axis=0)
        boot_diffs[i, :] = boot_pips - boot_lo
    
    # Calculate confidence intervals
    ci_lower = np.percentile(boot_diffs, 2.5, axis=0)
    ci_upper = np.percentile(boot_diffs, 97.5, axis=0)
    
    # Identify significant differences (95% CI doesn't cross zero)
    sig_boot = (ci_lower > 0) | (ci_upper < 0)
    sig_boot_count = np.sum(sig_boot)
    print(f"Found {sig_boot_count} ROIs with significant differences via bootstrap")
    
    # Leave-One-Out Cross-Validation for reliability
    print("Performing Leave-One-Out Cross-Validation...")
    loo_reliability = np.zeros(n_masked_rois)
    
    for left_out in range(n_valid_subjects):
        # Create the training set (all subjects except the left out one)
        train_idx = list(range(n_valid_subjects))
        train_idx.remove(left_out)
        
        # Calculate mean training differences
        train_diff = np.mean(pips_fc_masked[train_idx, :] - lo_fc_masked[train_idx, :], axis=0)
        
        # Test on left out subject
        test_diff = pips_fc_masked[left_out, :] - lo_fc_masked[left_out, :]
        
        # Calculate consistency of signs between training and test
        loo_reliability += (np.sign(train_diff) == np.sign(test_diff)).astype(float)
    
    # Convert to proportion of consistent predictions
    loo_reliability = loo_reliability / n_valid_subjects
    
    # Set threshold at 75% consistency
    loo_threshold = 0.75
    sig_loo = loo_reliability >= loo_threshold
    sig_loo_count = np.sum(sig_loo)
    
    print(f"LOO reliability threshold: {loo_threshold:.3f}")
    print(f"Found {sig_loo_count} ROIs with high reliability via LOO")
    
    # Combine significance
    sig_combined = sig_boot & sig_loo
    sig_combined_count = np.sum(sig_combined)
    print(f"Found {sig_combined_count} ROIs significant with combined criteria")
    
    # Helper function to get ROI name
    def get_roi_name(roi_id):
        try:
            idx = int(roi_id) - 1
            if 0 <= idx < len(atlas_labels):
                label = atlas_labels[idx]
                if isinstance(label, bytes):
                    label = label.decode('utf-8')
                return label
        except:
            pass
        return f"ROI_{roi_id}"
    
    # Create a map from masked indices to original atlas indices
    original_indices = np.where(mask)[0]
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'ROI_ID': original_indices + 1,  # Convert to 1-based indexing
        'ROI_Name': [get_roi_name(i+1) for i in original_indices],
        'pIPS_Connectivity': mean_pips_masked,
        'LO_Connectivity': mean_lo_masked,
        'Difference': diff_profile,
        'Abs_Difference': abs_diff_profile,
        'CI_Lower': ci_lower,
        'CI_Upper': ci_upper,
        'LOO_Reliability': loo_reliability,
        'Bootstrap_Significant': sig_boot,
        'LOO_Significant': sig_loo,
        'Combined_Significant': sig_combined,
        'Direction': np.where(diff_profile > 0, 'pIPS > LO', 'LO > pIPS')
    })
    
    # Save results to CSV
    csv_path = f'{output_dir}/bilateral_{analysis_type}_connectivity_fingerprint_results.csv'
    results_df.to_csv(csv_path, index=False)
    print(f"Results saved to {csv_path}")
    
    # Create roi_data dictionary for visualization
    roi_data = {
        'mean_pips': mean_pips_masked,
        'mean_lo': mean_lo_masked,
        'diff_profile': diff_profile,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'sig_combined': sig_combined
    }
    
    return results_df, roi_data

# Run analysis for FC
fc_results, fc_data = analyze_connectivity_and_save_results(analysis_type='fc')

### 3. Statistical Analysis of Connectivity Asymmetry

To further characterize the differences between dorsal and ventral connectivity profiles, we performed statistical analyses comparing the number and strength of connections for each pathway.

The connectivity asymmetry analysis revealed a striking imbalance in the number of connections favoring different pathways:

- pIPS showed stronger connectivity with 50 regions, while LO showed stronger connectivity with only 10 regions
- This asymmetry was highly significant (χ² = 26.67, p < 0.001)
- Mean strength of connections did not differ significantly (pIPS: 0.0359 ± 0.0200, LO: 0.0266 ± 0.0128, p = 0.30)
- Total connectivity strength was substantially higher for pIPS (pIPS/LO ratio: 6.76)

These results support our hypotheses that pIPS (dorsal pathway) maintains significantly more connections throughout the brain compared to LO (ventral pathway), though the strength of individual connections is similar.

In [ ]:
# Analyze dorsal-ventral asymmetry in connectivity
def analyze_connectivity_asymmetry(analysis_type='fc'):
    """
    Analyze the asymmetry in number and strength of connections for pIPS and LO regions.
    Tests whether pIPS has significantly more and stronger connections than LO.
    
    Parameters:
    -----------
    analysis_type : str
        Type of analysis to perform ('fc' for functional connectivity)
        
    Returns:
    --------
    dict
        Dictionary containing results of the asymmetry analysis
    """
    # Define paths
    results_dir = '/user_data/csimmon2/git_repos/ptoc/results'
    connectivity_dir = f'{results_dir}/connectivity_comparison'
    output_dir = f'{results_dir}/connectivity_asymmetry'
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"\n{'='*80}")
    print(f"ANALYZING {analysis_type.upper()} ASYMMETRY IN CONNECTIONS BETWEEN pIPS AND LO...")
    print(f"{'='*80}")
    
    # Load connectivity results from CSV
    results_path = f'{connectivity_dir}/bilateral_{analysis_type}_connectivity_fingerprint_results.csv'
    if not os.path.exists(results_path):
        print(f"Error: Could not find {analysis_type.upper()} connectivity results at {results_path}")
        return None
    
    results_df = pd.read_csv(results_path)
    print(f"Loaded {analysis_type.upper()} connectivity data with {len(results_df)} ROIs")
    
    # Filter to significant regions according to the combined criteria
    sig_regions = results_df[results_df['Combined_Significant'] == True]
    print(f"Found {len(sig_regions)} significant ROIs")
    
    # Count regions by direction
    pips_stronger_count = sum(sig_regions['Direction'] == 'pIPS > LO')
    lo_stronger_count = sum(sig_regions['Direction'] == 'LO > pIPS')
    
    print(f"pIPS > LO: {pips_stronger_count} regions")
    print(f"LO > pIPS: {lo_stronger_count} regions")
    
    # 1. Chi-square test for number of connections
    observed = np.array([pips_stronger_count, lo_stronger_count])
    expected = np.array([observed.sum()/2, observed.sum()/2])  # Expected under null hypothesis of equal distribution
    
    chi2, p_value = stats.chisquare(observed, expected)
    
    print("\nChi-square test for asymmetry in number of connections:")
    print(f"Observed frequencies: pIPS > LO: {observed[0]}, LO > pIPS: {observed[1]}")
    print(f"Expected frequencies: pIPS > LO: {expected[0]}, LO > pIPS: {expected[1]}")
    print(f"Chi-square statistic: {chi2:.4f}")
    print(f"p-value: {p_value:.8f}")
    
    if p_value < 0.05:
        print("The asymmetry in number of connections is statistically significant.")
        if pips_stronger_count > lo_stronger_count:
            print(f"pIPS has significantly more connections than LO (p {'< .001' if p_value < 0.001 else f'= {p_value:.4f}'}).")
        else:
            print(f"LO has significantly more connections than pIPS (p {'< .001' if p_value < 0.001 else f'= {p_value:.4f}'}).")
    else:
        print("The asymmetry in number of connections is not statistically significant.")
    
    # 2. Extract connectivity strength values for significant regions
    pips_stronger = np.abs(sig_regions[sig_regions['Direction'] == 'pIPS > LO']['Difference'].values)
    lo_stronger = np.abs(sig_regions[sig_regions['Direction'] == 'LO > pIPS']['Difference'].values)
    
    pips_mean = np.mean(pips_stronger) if len(pips_stronger) > 0 else 0
    pips_std = np.std(pips_stronger) if len(pips_stronger) > 0 else 0
    lo_mean = np.mean(lo_stronger) if len(lo_stronger) > 0 else 0
    lo_std = np.std(lo_stronger) if len(lo_stronger) > 0 else 0
    
    print(f"\nMean strength of pIPS-dominant connections: {pips_mean:.4f} ± {pips_std:.4f}")
    print(f"Mean strength of LO-dominant connections: {lo_mean:.4f} ± {lo_std:.4f}")
    
    # Mann-Whitney U test for comparing strength distributions (better for unequal sample sizes)
    u_stat = None
    p_value_mw = None
    
    if len(pips_stronger) > 0 and len(lo_stronger) > 0:
        u_stat, p_value_mw = stats.mannwhitneyu(pips_stronger, lo_stronger, alternative='two-sided')
        
        print("\nMann-Whitney U test for connection strength differences:")
        print(f"U statistic: {u_stat}")
        print(f"p-value: {p_value_mw:.8f}")
        
        if p_value_mw < 0.05:
            if np.median(pips_stronger) > np.median(lo_stronger):
                print(f"pIPS-dominant connections are significantly stronger than LO-dominant connections (p {'< .001' if p_value_mw < 0.001 else f'= {p_value_mw:.4f}'}).")
            else:
                print(f"LO-dominant connections are significantly stronger than pIPS-dominant connections (p {'< .001' if p_value_mw < 0.001 else f'= {p_value_mw:.4f}'}).")
        else:
            print("No significant difference in connection strength between pIPS-dominant and LO-dominant regions.")
        
        # Also calculate total connectivity strength
        pips_total = np.sum(pips_stronger)
        lo_total = np.sum(lo_stronger)
        print("\nTotal connectivity strength:")
        print(f"pIPS: {pips_total:.4f} (across {len(pips_stronger)} connections)")
        print(f"LO: {lo_total:.4f} (across {len(lo_stronger)} connections)")
        if lo_total > 0:
            print(f"Ratio pIPS/LO: {pips_total/lo_total:.2f}")
    
    # Create visualizations
    plt.figure(figsize=(15, 5))
    
    # Plot number of connections
    plt.subplot(1, 3, 1)
    plt.bar(['pIPS > LO', 'LO > pIPS'], [pips_stronger_count, lo_stronger_count], 
            color=['#4ac0c0', '#ff9b83'])
    plt.title(f'{analysis_type.upper()}: Number of Significant Connections')
    plt.ylabel('Count')
    
    # Plot connection strengths with boxplot if both groups have data
    if len(pips_stronger) > 0 and len(lo_stronger) > 0:
        plt.subplot(1, 3, 2)
        boxplot = plt.boxplot([pips_stronger, lo_stronger], labels=['pIPS > LO', 'LO > pIPS'], 
                  patch_artist=True)
        
        # Set colors to match the bar plot
        boxplot['boxes'][0].set_facecolor('#4ac0c0')
        boxplot['boxes'][1].set_facecolor('#ff9b83')
        
        plt.title(f'{analysis_type.upper()}: Connection Strength Distribution')
        plt.ylabel('Absolute Connectivity Difference')
        
        # Add median lines to the plot
        plt.axhline(y=np.median(pips_stronger), color='#4ac0c0', linestyle='--', alpha=0.5)
        plt.axhline(y=np.median(lo_stronger), color='#ff9b83', linestyle='--', alpha=0.5)
        
        # Plot violin plot to better visualize distributions
        plt.subplot(1, 3, 3)
        violin = plt.violinplot([pips_stronger, lo_stronger], showmeans=True)
        
        # Customize violin plot colors
        for i, pc in enumerate(violin['bodies']):
            if i == 0:
                pc.set_facecolor('#4ac0c0')
            else:
                pc.set_facecolor('#ff9b83')
        
        plt.xticks([1, 2], ['pIPS > LO', 'LO > pIPS'])
        plt.title(f'{analysis_type.upper()}: Distribution of Connection Strengths')
        plt.ylabel('Absolute Connectivity Difference')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/{analysis_type}_connectivity_asymmetry_analysis.png', dpi=300)
    
    # Calculate median values safely
    pips_median = np.median(pips_stronger) if len(pips_stronger) > 0 else None
    lo_median = np.median(lo_stronger) if len(lo_stronger) > 0 else None
    
    return {
        'analysis_type': analysis_type,
        'number_test': {'chi2': chi2, 'p_value': p_value},
        'strength_test': {'u_stat': u_stat, 'p_value': p_value_mw},
        'counts': {'pips_stronger': pips_stronger_count, 'lo_stronger': lo_stronger_count},
        'strengths': {'pips_mean': pips_mean,
                     'lo_mean': lo_mean,
                     'pips_median': pips_median,
                     'lo_median': lo_median,
                     'pips_total': np.sum(pips_stronger),
                     'lo_total': np.sum(lo_stronger)}
    }

# Run asymmetry analysis for FC
asymmetry_results = analyze_connectivity_asymmetry(analysis_type='fc')

In [ ]:
# Visualize Brain Connectivity
def visualize_brain_connectivity(analysis_type='fc'):
    """
    Create brain visualization for specified data type using saved CSV results
    
    Parameters:
    -----------
    analysis_type : str
        Type of analysis to visualize ('fc' for functional connectivity)
        
    Returns:
    --------
    pandas.DataFrame
        DataFrame containing significant ROIs
    """
    print(f"\n{'='*80}")
    print(f"CREATING {analysis_type.upper()} BRAIN CONNECTIVITY FIGURE FROM SAVED DATA")
    print(f"{'='*80}")
    
    # Load merged atlas instead of the standard Schaefer atlas
    merged_atlas_file = f'{results_dir}/schaefer_wang_merged.nii.gz'
    merged_labels_file = f'{results_dir}/merged_atlas_labels.npy'
    
    if not os.path.exists(merged_atlas_file) or not os.path.exists(merged_labels_file):
        print("Error: Merged atlas files not found. Please run merge_atlas.py first.")
        return None
    
    # Load merged atlas
    atlas_img = nib.load(merged_atlas_file)
    atlas_labels = np.load(merged_labels_file, allow_pickle=True)
    print(f"Loaded merged atlas with {len(atlas_labels)} regions")
    
    # Load saved connectivity results
    csv_path = f'{connectivity_dir}/bilateral_{analysis_type}_connectivity_fingerprint_results.csv'
    if not os.path.exists(csv_path):
        print(f"Error: Could not find {analysis_type.upper()} results at {csv_path}")
        return None
    
    # Load the data
    results_df = pd.read_csv(csv_path)
    print(f"Loaded {analysis_type.upper()} data with {len(results_df)} ROIs")
    
    # Filter to significant regions according to the combined criteria
    sig_df = results_df[results_df['Combined_Significant'] == True].copy()
    sig_df['Hemisphere'] = sig_df['ROI_Name'].apply(lambda x: 'L' if 'LH' in x else 'R' if 'RH' in x else 'X')
    
    print(f"Found {len(sig_df)} significant ROIs")
    
    if len(sig_df) > 0:
        # Get coordinates for all ROIs from the merged atlas
        all_coords = plotting.find_parcellation_cut_coords(atlas_img)
        
        # Prepare node properties for significant ROIs
        sig_coords = []
        sig_colors = []
        sig_sizes = []
        
        for _, row in sig_df.iterrows():
            roi_idx = row['ROI_ID'] - 1  # Convert to 0-based index
            sig_coords.append(all_coords[roi_idx])
            sig_colors.append('#4ac0c0' if row['Difference'] > 0 else '#ff9b83')
            sig_sizes.append(abs(row['Difference']) * 40 + 30)
        
        # Add seed nodes
        seed_coords = [
            [-26, -68, 48], [26, -68, 48],  # pIPS L/R
            [-42, -82, -8], [42, -82, -8]    # LO L/R
        ]
        all_coords = sig_coords + seed_coords
        all_colors = sig_colors + ['#4ac0c0', '#4ac0c0', '#ff9b83', '#ff9b83']
        all_sizes = sig_sizes + [200, 200, 200, 200]
        
        # Create adjacency matrix
        adjacency = np.zeros((len(all_coords), len(all_coords)))
        for i, row in enumerate(sig_df.itertuples()):
            tgt_offset = 0 if row.Difference > 0 else 2
            tgt_idx = len(sig_coords) + tgt_offset + (0 if row.Hemisphere == 'L' else 1)
            adjacency[i, tgt_idx] = adjacency[tgt_idx, i] = 1
        
        # Create network plot
        fig = plt.figure(figsize=(18, 16))
        plotting.plot_connectome(
            adjacency, all_coords,
            node_color=all_colors,
            node_size=all_sizes,
            edge_kwargs={'color': 'gray', 'alpha': 0.3, 'linewidth': 1.0},
            display_mode='lzry',
            title=f'{analysis_type.upper()}: All {len(sig_df)} Significant ROIs',
            figure=fig
        )
        
        # Save the figure
        output_path = f'{connectivity_dir}/{analysis_type}_brain_connectivity.png'
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        
        print(f"Brain network visualization complete for {analysis_type}")
        print(f"Showing {len(sig_df)} significant ROIs connected to pIPS and LO seed regions")
        print(f"Figure saved to {output_path}")
    else:
        print(f"No significant ROIs found for {analysis_type}. Cannot create brain visualization.")
    
    return sig_df

# Create brain connectivity visualization
sig_roi_df = visualize_brain_connectivity(analysis_type='fc')

In [ ]:
# Visualize Anatomical Organization
def visualize_anatomical_organization(results_df, roi_data, analysis_type='fc'):
    """
    Create anatomical organization visualization
    
    Parameters:
    -----------
    results_df : pandas.DataFrame
        DataFrame containing connectivity results
    roi_data : dict
        Dictionary containing ROI data for visualization
    analysis_type : str
        Type of analysis to visualize ('fc' for functional connectivity)
    """
    print(f"\n{'='*80}")
    print(f"CREATING {analysis_type.upper()} ANATOMICAL ORGANIZATION VISUALIZATION")
    print(f"{'='*80}")
    
    # Extract data from parameters
    mean_pips = roi_data['mean_pips']
    mean_lo = roi_data['mean_lo']
    diff_profile = roi_data['diff_profile']
    ci_lower = roi_data['ci_lower']
    ci_upper = roi_data['ci_upper']
    sig_combined = roi_data['sig_combined']
    
    # Function to map ROI names to anatomical lobes
    def map_to_anatomical_lobe(roi_name):
        # Special handling for Wang atlas ROIs
        if 'Wang_pIPS' in roi_name:
            return 'Parietal'
        elif 'Wang_LO' in roi_name:
            return 'Temporal'
        
        # Map Schaefer networks to anatomical regions
        if 'Vis' in roi_name:
            return 'Occipital'
        elif 'SomMot' in roi_name:
            return 'Somatomotor'
        elif 'DorsAttn' in roi_name:
            if 'Par' in roi_name or 'IPL' in roi_name or 'IPS' in roi_name:
                return 'Parietal'
            elif 'Temp' in roi_name or 'MT' in roi_name:
                return 'Temporal'
            else:
                return 'Parietal'
        elif 'SalVentAttn' in roi_name:
            if 'Ins' in roi_name:
                return 'Insular'
            elif 'Cing' in roi_name or 'ACC' in roi_name:
                return 'Cingulate'
            elif 'Temp' in roi_name:
                return 'Temporal'
            elif 'Par' in roi_name:
                return 'Parietal'
            else:
                return 'Frontal'
        elif 'Limbic' in roi_name:
            if 'Temp' in roi_name:
                return 'Temporal'
            else:
                return 'Frontal'
        elif 'Cont' in roi_name:
            if 'Par' in roi_name or 'IPL' in roi_name or 'IPS' in roi_name:
                return 'Parietal'
            elif 'Temp' in roi_name or 'MT' in roi_name:
                return 'Temporal'
            elif 'Cing' in roi_name:
                return 'Cingulate'
            else:
                return 'Frontal'
        elif 'Default' in roi_name:
            if 'Par' in roi_name:
                return 'Parietal'
            elif 'Temp' in roi_name:
                return 'Temporal'
            elif 'PCC' in roi_name or 'Cing' in roi_name:
                return 'Cingulate'
            else:
                return 'Frontal'
        else:
            return 'Other'
    
    # Clean up ROI names for better labeling
    def clean_roi_name(roi_name):
        if isinstance(roi_name, bytes):
            roi_name = roi_name.decode('utf-8')
        
        # Handle Wang ROIs
        if 'Wang_' in roi_name:
            return roi_name.replace('Wang_', '')
        
        # Remove common prefixes and suffixes
        cleaned = roi_name.replace('7Networks_', '')
        
        # Extract just the region part, removing hemisphere and network
        parts = cleaned.split('_')
        if len(parts) > 2:
            return parts[-2] + '_' + parts[-1]
        elif len(parts) > 1:
            return parts[-1]
        else:
            return cleaned
    
    # Determine hemisphere and base region for organizing pairs
    def get_hemisphere_and_region(roi_name):
        if isinstance(roi_name, bytes):
            roi_name = roi_name.decode('utf-8')
        
        # Determine hemisphere 
        if 'LH' in roi_name:
            hemisphere = 'L'
        elif 'RH' in roi_name:
            hemisphere = 'R'
        else:
            # For Wang ROIs or ROIs without clear hemisphere
            if 'Wang_pIPS' in roi_name or 'Wang_LO' in roi_name:
                hemisphere = 'X'  # Bilateral
            else:
                hemisphere = 'X'  # Unknown
        
        # Extract base region by removing hemisphere and number
        cleaned = roi_name.replace('7Networks_', '')
        cleaned = cleaned.replace('LH_', '').replace('RH_', '')
        
        # Handle Wang ROIs
        if 'Wang_' in cleaned:
            base_region = cleaned.replace('Wang_', '')
        else:
            # Remove trailing numbers which often differentiate regions
            import re
            base_region = re.sub(r'_\d+$', '', cleaned)
        
        return hemisphere, base_region
    
    # Add anatomical lobe and hemisphere information to the dataframe
    results_df['Anatomical_Lobe'] = results_df['ROI_Name'].apply(map_to_anatomical_lobe)
    results_df['Clean_Name'] = results_df['ROI_Name'].apply(clean_roi_name)
    
    # Extract hemisphere and base region info
    hemisphere_region = results_df['ROI_Name'].apply(get_hemisphere_and_region)
    results_df['Hemisphere'] = [h for h, r in hemisphere_region]
    results_df['Base_Region'] = [r for h, r in hemisphere_region]
    
    # Define a manual order for anatomical lobes
    lobe_order = [
        'Frontal',
        'Somatomotor',
        'Parietal',
        'Temporal',
        'Occipital',
        'Insular',
        'Cingulate',
        'Other'
    ]
    
    # Create a category for sorting based on the defined order
    from pandas.api.types import CategoricalDtype
    lobe_cat = pd.Categorical(results_df['Anatomical_Lobe'], categories=lobe_order, ordered=True)
    results_df['Lobe_Sorted'] = lobe_cat
    
    # Define a consistent color palette for brain lobes
    lobe_colors = {
        'Frontal': '#3498db',     # Blue
        'Somatomotor': '#f1c40f', # Yellow
        'Parietal': '#e74c3c',    # Red
        'Temporal': '#2ecc71',    # Green
        'Occipital': '#9b59b6',   # Purple
        'Insular': '#f39c12',     # Orange
        'Cingulate': '#1abc9c',   # Turquoise
        'Other': '#7f8c8d'        # Gray
    }
    
    # Custom sorting function to group by lobe, then by base region, then by hemisphere (L then R)
    def custom_sort(row):
        lobe_idx = lobe_order.index(row['Anatomical_Lobe']) if row['Anatomical_Lobe'] in lobe_order else 999
        hemi_idx = 0 if row['Hemisphere'] == 'L' else 1 if row['Hemisphere'] == 'R' else 2
        return (lobe_idx, row['Base_Region'], hemi_idx)
    
    # Sort using the custom function
    results_df['sort_key'] = results_df.apply(custom_sort, axis=1)
    results_df_sorted = results_df.sort_values('sort_key')
    
    # Get the new order
    sorted_indices = results_df_sorted.index.values
    
    # Extract reordered data
    mean_pips_sorted = results_df_sorted['pIPS_Connectivity'].values
    mean_lo_sorted = results_df_sorted['LO_Connectivity'].values
    diff_profile_sorted = results_df_sorted['Difference'].values
    sig_sorted = results_df_sorted['Combined_Significant'].values
    
    # Find peak (highest) and bottom (lowest) significant ROI in each lobe
    significant_peak_bottom_indices = []
    lobe_groups = results_df_sorted.groupby('Anatomical_Lobe')
    
    for lobe, group in lobe_groups:
        if len(group) > 0:
            # Get significant ROIs in this lobe
            sig_group = group[group['Combined_Significant']]
            
            if len(sig_group) > 0:
                # Get highest difference (most pIPS-preferring) significant ROI
                peak_idx = sig_group['Difference'].idxmax()
                significant_peak_bottom_indices.append(peak_idx)
                
                # Get lowest difference (most LO-preferring) significant ROI
                bottom_idx = sig_group['Difference'].idxmin()
                significant_peak_bottom_indices.append(bottom_idx)
    
    # Create the visualization
    plt.figure(figsize=(18, 10))
    
    # Top subplot - connectivity profiles with lollipop plot, organized by lobe
    ax1 = plt.subplot(2, 1, 1)
    x = np.arange(len(mean_pips_sorted))
    
    # Get the y-limits for label positioning
    y_min, y_max = min(min(mean_pips_sorted), min(mean_lo_sorted)), max(max(mean_pips_sorted), max(mean_lo_sorted))
    y_range = y_max - y_min
    
    # Plot background colors for different lobes
    current_lobe = None
    start_idx = 0
    
    for i, idx in enumerate(sorted_indices):
        lobe = results_df_sorted.iloc[i]['Anatomical_Lobe']
        if lobe != current_lobe:
            if current_lobe is not None:
                # Mark the region for the previous lobe
                plt.axvspan(start_idx - 0.5, i - 0.5, alpha=0.15, 
                          color=lobe_colors[current_lobe])
                # Add label in the middle of the region with fixed top position
                label_y = y_max + 0.15 * y_range  # Position labels above the data
                plt.text((start_idx + i - 1) / 2, label_y, 
                       current_lobe, ha='center', fontsize=11, fontweight='bold')
            current_lobe = lobe
            start_idx = i
    
    # Add the last region
    plt.axvspan(start_idx - 0.5, len(mean_pips_sorted) - 0.5, alpha=0.15, 
              color=lobe_colors[current_lobe])
    plt.text((start_idx + len(mean_pips_sorted) - 1) / 2, y_max + 0.15 * y_range, 
           current_lobe, ha='center', fontsize=11, fontweight='bold')
    
    # Create lollipop plot - vertical lines connecting pIPS and LO points
    for i in range(len(x)):
        plt.plot([x[i], x[i]], [mean_pips_sorted[i], mean_lo_sorted[i]], color='gray', linestyle='-', linewidth=0.5)
    
    # Plot the points
    plt.scatter(x, mean_pips_sorted, color='#4ac0c0', s=20, label='pIPS')
    plt.scatter(x, mean_lo_sorted, color='#ff9b83', s=20, label='LO')
    
    # Set y-axis limits with padding for the labels
    plt.ylim(y_min - 0.1 * y_range, y_max + 0.25 * y_range)
    
    plt.ylabel('Connectivity Strength')
    plt.title(f'{analysis_type.upper()} ROI Connectivity Profiles (Organized by Anatomical Regions, L-R Pairs)')
    plt.legend(loc='upper left')
    plt.xlim(-0.5, len(mean_pips_sorted) - 0.5)
    
    # Bottom subplot - difference
    ax2 = plt.subplot(2, 1, 2)
    
    # Create base bar plot for all bars
    base_colors = ['#4ac0c0' if val > 0 else '#ff9b83' for val in diff_profile_sorted]
    bars = plt.bar(x, diff_profile_sorted, color=base_colors)
    
    # Apply transparency to non-significant bars individually
    for i, (bar, is_sig) in enumerate(zip(bars, sig_sorted)):
        if not is_sig:
            # Set alpha for non-significant bars only
            bar.set_alpha(0.3)
    
    plt.axhline(y=0, color='black', linestyle='-')
    
    # Add a legend for significance
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor='gray', alpha=1.0, label='Significant'),
        Patch(facecolor='gray', alpha=0.3, label='Non-significant')
    ]
    plt.legend(handles=legend_elements, loc='upper right')
    
    # Add background colors for different lobes (same as top plot)
    current_lobe = None
    start_idx = 0
    
    for i, idx in enumerate(sorted_indices):
        lobe = results_df_sorted.iloc[i]['Anatomical_Lobe']
        if lobe != current_lobe:
            if current_lobe is not None:
                plt.axvspan(start_idx - 0.5, i - 0.5, alpha=0.15, 
                          color=lobe_colors[current_lobe])
            current_lobe = lobe
            start_idx = i
    
    # Add the last region
    plt.axvspan(start_idx - 0.5, len(mean_pips_sorted) - 0.5, alpha=0.15, 
              color=lobe_colors[current_lobe])
    
    # Label only significant peak and bottom ROIs
    for peak_idx in significant_peak_bottom_indices:
        # Find position in sorted array
        position = np.where(sorted_indices == peak_idx)[0][0]
        roi_id = results_df.loc[peak_idx, 'ROI_ID']
        roi_name = results_df.loc[peak_idx, 'Clean_Name']
        diff_value = results_df.loc[peak_idx, 'Difference']
        
        # Add annotation for ROI
        plt.annotate(f'ROI {roi_id}\n{roi_name}',
                   xy=(position, diff_value),
                   xytext=(0, 20 if diff_value >= 0 else -25),
                   textcoords='offset points',
                   ha='center',
                   va='bottom' if diff_value >= 0 else 'top',
                   fontsize=8,
                   fontweight='bold',
                   bbox=dict(boxstyle='round,pad=0.3', fc='white', alpha=0.8),
                   arrowprops=dict(arrowstyle='->', lw=1.5))
    
    plt.ylabel('pIPS - LO Difference')
    plt.xlabel('ROI ID (Organized by Anatomical Region, L-R Pairs)')
    plt.title(f'{analysis_type.upper()} Connectivity Difference (Transparent bars = non-significant)')
    plt.xlim(-0.5, len(mean_pips_sorted) - 0.5)
    plt.ylim(-0.2, 0.2)  # For FC analysis range from -0.2 to 0.2
    
    plt.tight_layout()
    
    # Save the figure
    fig_path = f'{output_dir}/{analysis_type}_anatomical_organization.png'
    plt.savefig(fig_path, dpi=300)
    
    print(f"Figure saved to {fig_path}")
    
    # Print peak and bottom ROIs by lobe, only for significant ROIs
    print(f"\n{analysis_type.upper()} significant peak and bottom ROIs by anatomical lobe:")
    for lobe, group in lobe_groups:
        # Get significant ROIs in this lobe
        sig_group = group[group['Combined_Significant']]
        
        if len(sig_group) > 0:
            print(f"\n{lobe} Lobe:")
            
            # Peak (pIPS-preferring) significant ROI
            peak_row = sig_group.loc[sig_group['Difference'].idxmax()]
            print(f"  Peak (pIPS > LO): ROI {peak_row['ROI_ID']} ({peak_row['Clean_Name']}), " + 
                 f"Diff = {peak_row['Difference']:.3f}")
            
            # Bottom (LO-preferring) significant ROI
            bottom_row = sig_group.loc[sig_group['Difference'].idxmin()]
            print(f"  Bottom (LO > pIPS): ROI {bottom_row['ROI_ID']} ({bottom_row['Clean_Name']}), " + 
                 f"Diff = {bottom_row['Difference']:.3f}")

# Create anatomical organization visualization
visualize_anatomical_organization(fc_results, fc_data, analysis_type='fc')

## Conclusions

Our findings provide robust evidence for an integrated network architecture supporting object recognition, with distinct connectivity profiles for dorsal (pIPS) and ventral (LO) visual pathways. The remarkably high spatial overlap (Dice coefficient = 0.91) between these pathways within individual subjects indicates they function as a singular network rather than separate systems. Despite this integration, we identified significant asymmetry in connectivity patterns, with the dorsal pathway showing stronger connectivity with five times more brain regions than the ventral pathway, though individual connection strengths remained similar. This pattern supports a model where the dorsal "where/how" pathway has broader influence across brain networks during object processing, while the ventral "what" pathway maintains more focused connectivity aligned with its specialized role in object recognition. These findings challenge traditional views of strictly segregated visual pathways and advance our understanding of how the brain integrates object recognition with spatial and action-related processing.